In [1]:
import numpy as np
import os
import pandas as pd
import xml.etree.ElementTree as et

# %load_ext nb_black

In [2]:
# UCD_trn xml file path
extra_xml_path = r"D:\tencent_files\chrome_Download\Research\Equilibium_modelling\Partial_equilibium\GCAM\gcam-v5.2\input\extra"
dir_extra = os.listdir(extra_xml_path)  # SSP3 uses trn_UCD_SSP3
trn_xml_ls = [file for file in dir_extra if "transportation_UCD_SSP" in file and "_new" not in file]
trn_xml_path = dict(
    zip(
        ["ssp" + str(i) for i in [1, 2, 4, 5]],
        [os.path.join(extra_xml_path, file) for file in trn_xml_ls],
    )
)

In [3]:
xml_path = r"D:\tencent_files\chrome_Download\Research\Equilibium_modelling\Partial_equilibium\GCAM\gcam-v5.2\input\gcamdata\xml"
hydro_xml = os.path.join(xml_path,"hydrogen.xml")
FCEVtruck_xml = os.path.join(xml_path,"transportation_UCD_FCEVtruck.xml")

# Transport cost revise

In [4]:
# Car type
tranSubsector_name_ls = [
    "Compact Car",
    "Large Car and SUV",
    "Mini Car",
    "Multipurpose Vehicle",
    "Subcompact Car",
]

In [5]:
node_tag_ls = ["minicam-non-energy-input","minicam-energy-input","loadFactor"]
subnode_tag_ls = ["input-cost","coefficient"]
supplysector_name_ls = ["trn_pass_road_LDV_4W","trn_freight_road"]

In [6]:
def xml_input_revise(path,
                   new_file_name,
                data_input,
#                 period_to_rev,
#                 input_cost_to_rev,
                supplysector_name_ls=supplysector_name_ls,
                  node_tag_ls = node_tag_ls,
                  subnode_tag_ls = subnode_tag_ls):
    data = data_input
    xtree = et.parse(path)
    xroot = xtree.getroot()
    rows = []
    for child in xroot:
        for region in child:
            if region.attrib.get("name") == "China":
                for supplysector in region:
                    if supplysector.attrib.get("name") in supplysector_name_ls: #supplysector_name
                        for tranSubsector in supplysector:
                            if tranSubsector.attrib.get("name") in data.index.unique(level="tranSubsector"):
                                for stubtechnology in tranSubsector:
                                    if stubtechnology.attrib.get("name") in data.index.unique("stubtechnology"):
                                        for period in stubtechnology:
                                            if period.attrib.get("year") in [str(y) for y in data.columns]:#period_to_rev:
                                                for node in period:
                                                    if node.tag in node_tag_ls:
                                                        try:
                                                            tranSubsector_name = tranSubsector.attrib.get("name")
                                                            stubtechnology_name = stubtechnology.attrib.get("name")
                                                            year = int(period.attrib.get("year"))
                                                            if node.tag =="loadFactor":
                                                                node.text = str(data.loc[(tranSubsector_name,
                                                                                                  stubtechnology_name,
                                                                                                     node.tag),
                                                                                                 year])
                                                            else:
                                                                for sub_node in node:
                                                                    if (
                                                                        sub_node.tag
                                                                        in subnode_tag_ls
                                                                    ):

                                                                        sub_node.text = str(data.loc[(tranSubsector_name,
                                                                                                  stubtechnology_name,
                                                                                                     sub_node.tag),
                                                                                                 year])
                                                        except:
                                                            pass

    xtree.write(new_file_name,encoding="UTF-8",xml_declaration=True)

In [8]:
scenario_data_path = r"D:\tencent_files\chrome_Download\Research\Equilibium_modelling\H2MetalDemand\Data Collection\Scenario_data"

LDV_input_cost = pd.read_excel(os.path.join(scenario_data_path,"H2Tec_database.xlsx"), 
                                sheet_name = "H2_trn_LDV_cost",
                                index_col=[0,1,2])
LDV_input_cost.head()

2020      2025      2030  \
tranSubsector  stubtechnology tag                                        
Mini Car       BEV            input-cost  0.096268  0.087268  0.078268   
               FCEV           input-cost  0.108783  0.098953  0.089123   
Subcompact Car BEV            input-cost  0.156262  0.139902  0.123542   
               FCEV           input-cost  0.156973  0.143873  0.130773   
Compact Car    BEV            input-cost  0.233322  0.215089  0.196856   

                                              2035      2040      2045  \
tranSubsector  stubtechnology tag                                        
Mini Car       BEV            input-cost  0.069299  0.068933  0.068567   
               FCEV           input-cost  0.079253  0.079020  0.078787   
Subcompact Car BEV            input-cost  0.107196  0.106596  0.105996   
               FCEV           input-cost  0.117650  0.117250  0.116850   
Compact Car    BEV            input-cost  0.178624  0.177658  0.176692   

                                              2050      2055      2060  \
tranSubsector  stubtechnology tag                                        
Mini Car       BEV            input-cost  0.068188  0.068088  0.067988   
               FCEV           input-cost  0.078598  0.078598  0.078598   
Subcompact Car BEV            input-cost  0.105412  0.105212  0.105012   
               FCEV           input-cost  0.116515  0.116515  0.116515   
Compact Car    BEV            input-cost  0.175653  0.175353  0.175053   

                                              2065      2070      2075  \
tranSubsector  stubtechnology tag                                        
Mini Car       BEV            input-cost  0.067862  0.067729  0.067596   
               FCEV           input-cost  0.078524  0.078524  0.078524   
Subcompact Car BEV            input-cost  0.104846  0.104680  0.104514   
               FCEV           input-cost  0.116393  0.116393  0.116393   
Compact Car    BEV            input-cost  0.174812  0.174546  0.174280   

                                              2080      2085      2090  \
tranSubsector  stubtechnology tag                                        
Mini Car       BEV            input-cost  0.067538  0.067400  0.067300   
               FCEV           input-cost  0.078450  0.078450  0.078450   
Subcompact Car BEV            input-cost  0.104283  0.104083  0.103883   
               FCEV           input-cost  0.116271  0.116271  0.116200   
Compact Car    BEV            input-cost  0.173977  0.173677  0.173377   

                                              2095      2100  
tranSubsector  stubtechnology tag                             
Mini Car       BEV            input-cost  0.067216  0.067100  
               FCEV           input-cost  0.078377  0.078377  
Subcompact Car BEV            input-cost  0.103724  0.103524  
               FCEV           input-cost  0.116150  0.116150  
Compact Car    BEV            input-cost  0.173148  0.172848

In [9]:
xml_input_revise(trn_xml_path["ssp2"],
              os.path.join(extra_xml_path,'transportation_UCD_SSP2_new.xml'),
                LDV_input_cost,
                 node_tag_ls = node_tag_ls[0]
              )

# Transport adding FCEV Truck


## Read inputs from Liquid Truck

In [26]:
# Truck type
truck_tranSubsector_name_ls = [
"Truck (>14t)",
"Truck (0-6t)",
"Truck (6-14t)",
]

In [29]:
def xml_input_read(UCD_xml_path,
                    tranSubsector_name,
                    supplysector_name="trn_freight_road",
                   stubtechnology_name="Liquids",
                   node_tag = "minicam-energy-input",
                  subnode_tag="coefficient"):
    xtree = et.parse(UCD_xml_path)
    xroot  = xtree.getroot()
    rows = []
    for child in xroot:
        for region in child:
            if region.attrib.get("name") == "China":
                for supplysector in region:
                    if supplysector.attrib.get("name") in supplysector_name:
                        for tranSubsector in supplysector:
                            if tranSubsector.attrib.get("name") == tranSubsector_name:
                                for stubtechnology in tranSubsector:
                                    if stubtechnology.attrib.get("name") == stubtechnology_name:
                                        for period in stubtechnology:
                                            for node in period:
                                                if node.tag == node_tag:
                                                    s_region = region.attrib.get("name")
                                                    s_supplysector = supplysector.attrib.get("name")
                                                    s_tranSubsector = tranSubsector.attrib.get("name")
                                                    s_stubtechnology = stubtechnology.attrib.get("name")
                                                    s_period = period.attrib.get("year")
                                                    if node.tag =="loadFactor":
                                                        s_coefficient = (
                                                                    float(node.text)  
                                                                )
                                                        s_tag = node.tag
                                                        rows.append(
                                                                    dict(
                                                                        region=s_region,
                                                                        supplysector=s_supplysector,
                                                                        tranSubsector=s_tranSubsector,
                                                                        stubtechnology=s_stubtechnology,
                                                                        period=s_period,
                                                                        tag=s_tag,
                                                                        value=s_coefficient,
                                                                    )
                                                                )
                                                    else:
                                                        for sub_node in node:
                                                            if (
                                                                sub_node.tag
                                                                == subnode_tag
                                                            ):
                                                                
                                                                s_tag = sub_node.tag
                                                                s_coefficient = (
                                                                    float(sub_node.text)  
                                                                )
                                                                rows.append(
                                                                    dict(
                                                                        region=s_region,
                                                                        supplysector=s_supplysector,
                                                                        tranSubsector=s_tranSubsector,
                                                                        stubtechnology=s_stubtechnology,
                                                                        period=s_period,
                                                                        tag=s_tag,
                                                                        value=s_coefficient,
                                                                    )
                                                                )
    df = pd.DataFrame(rows)
    return df

In [27]:
def pivot_xml_input(ssp_trn_xml_path, 
                    truck_tranSubsector_name_ls=truck_tranSubsector_name_ls,
                    supplysector_name = "trn_freight_road",
                    stubtechnology_name = "Liquids",
                    node_tag = "minicam-energy-input",
                    subnode_tag = "input-cost"):
    value_ls = []
    for truck in truck_tranSubsector_name_ls:
        df = xml_input_read(ssp_trn_xml_path,
                                    truck,
                                    supplysector_name=supplysector_name,
                                    stubtechnology_name=stubtechnology_name,
                                    node_tag = node_tag,
                                    subnode_tag=subnode_tag )
        value_ls.append(df)

    value_df = pd.concat(value_ls)
    value_df["stubtechnology"] = "FCEV"
    value_df["period"] = value_df["period"].apply(lambda x: int(x))
    return value_df.pivot_table(columns="period",
                                index=("tranSubsector","stubtechnology","tag")
                                                       )["value"]

In [34]:
def ssp_truck_input(ssp_name):
    ssp_truck_energy = pivot_xml_input(os.path.join(xml_path,"transportation_UCD_CORE_5.1.xml"),
                                       stubtechnology_name = "FCEV",
                                       subnode_tag = "coefficient")
    ssp_truck_cost = pivot_xml_input(os.path.join(xml_path,"transportation_UCD_CORE_5.1.xml"),
                                     stubtechnology_name = "FCEV",
                                  node_tag = "minicam-non-energy-input",
                                  subnode_tag = "input-cost"
                                 )
    ssp_truck_loadFactor = pivot_xml_input(trn_xml_path[ssp_name], node_tag = "loadFactor")
#     ssp_LDV = pivot_xml_input(trn_xml_path[ssp_name], node_tag = "loadFactor")
#     ssp_truck_calibrated_value = pivot_xml_input(trn_xml_path[ssp_name],                             
#                                                  "Compact Car",
#                                                  supplysector_name="trn_pass_road_LDV_4W",
#                                                  stubtechnology_name="FCEV",
#                                                  node_tag = "minicam-energy-input",
#                                                  subnode_tag="calibrated-value"
#                                                 )
    ssp_truck = pd.concat([ssp_truck_cost,ssp_truck_energy,ssp_truck_loadFactor])
    xml_input_revise(FCEVtruck_xml,
                new_file_name=os.path.join(extra_xml_path,"transportation_UCD_"+ssp_name+"_FCEVtruck.xml"),
                data_input=ssp_truck,
                supplysector_name_ls = supplysector_name_ls,
                node_tag_ls = node_tag_ls,
                subnode_tag_ls = subnode_tag_ls
                )

In [1]:
    ssp_truck_energy = pivot_xml_input(os.path.join(xml_path,"transportation_UCD_CORE_5.1.xml"),
                                       stubtechnology_name = "FCEV",
                                       subnode_tag = "coefficient")
    ssp_truck_cost = pivot_xml_input(os.path.join(xml_path,"transportation_UCD_CORE_5.1.xml"),
                                       stubtechnology_name = "FCEV",
                                       node_tag = "minicam-non-energy-input",
                                       subnode_tag = "input-cost")
    ssp_truck_loadFactor = pivot_xml_input(trn_xml_path["ssp2"], 
                                           node_tag = "loadFactor")
    ssp_truck = pd.concat([ssp_truck_cost,ssp_truck_energy,ssp_truck_loadFactor])
    ssp_truck

NameError: name 'pivot_xml_input' is not defined

In [31]:
def main():
    for ssp_name in trn_xml_path.keys():
        ssp_truck_input(ssp_name)

In [35]:
main()

# Production cost revise

In [14]:
xtree = et.parse(hydro_xml)
xroot = xtree.getroot()

In [45]:
def H2_prod_cost(path,
                   new_file_name,
                cost_data,
#                 period_to_rev,
#                 input_cost_to_rev,
                supplysector_name =["H2 forecourt production", "H2 central production"]):
    data = cost_data
    xtree = et.parse(path)
    xroot = xtree.getroot()
    rows = []
    for child in xroot:
        for grandchild in child:
            if grandchild.tag == 'global-technology-database':
                for location_info in grandchild:                    
                    if (location_info.attrib.get("sector-name") in supplysector_name #supplysector_name "H2 forecourt production" "H2 central production"
                    and location_info.attrib.get("subsector-name") in data.index.unique("subsector")): # "electricity", "solar", 'wind'
                        for technology in location_info:
                            if technology.attrib.get("name") in data.index.unique("technology"):
                                for period in technology:
                                    if period.attrib.get("year") in [str(y) for y in data.columns]:#period_to_rev:
                                        for node in period:
                                            if node.tag == "minicam-non-energy-input":
                                                for sub_node in node:
                                                    if (
                                                        sub_node.tag
                                                        == "input-cost"
                                                    ):
                                                        sector_name = location_info.attrib.get("sector-name")
                                                        Subsector_name = location_info.attrib.get("subsector-name")
                                                        technology_name = technology.attrib.get("name")
                                                        year = int(period.attrib.get("year"))
                                                        sub_node.text = str(data.loc[(sector_name,
                                                                                      Subsector_name,
                                                                                      technology_name),
                                                                                    year])

    xtree.write(new_file_name,encoding="UTF-8",xml_declaration=True)

## RE electrolysis cost reduction

In [69]:
h2_prod_cost = pd.read_excel(os.path.join(data_path,"H2Tec_database.xlsx"), 
                       sheet_name = "H2_prod_cost",
                       index_col = [0,1,2])

In [70]:
h2_prod_cost.index[1]

('H2 central production', 'solar', 'electrolysis')

In [71]:
H2_prod_cost(path = hydro_xml,
            new_file_name=os.path.join(xml_path,"hydrogen_electrolysis.xml"),
            cost_data=h2_prod_cost)